In [159]:
import csv
from datetime import datetime

class Entry:
    def __init__(self, date, amount, description, entry_type):
        self.date = date
        self.amount = amount
        self.description = description
        self.entry_type = entry_type

    def to_list(self):
        return [self.date, self.amount, self.description, self.entry_type]

class DailyRecord:
    def __init__(self, date):
        self.date = date
        self.incomes = []
        self.expenses = []
    
    def add_income(self, amount, description):
        income = Entry(self.date, amount, description, 'income')
        self.incomes.append(income)
    
    def add_expense(self, amount, description):
        expense = Entry(self.date, amount, description, 'expense')
        self.expenses.append(expense)
    
    def get_summary(self):
        total_income = sum(income.amount for income in self.incomes)
        total_expense = sum(expense.amount for expense in self.expenses)
        balance = total_income - total_expense
        return {
            "date": self.date,
            "total_income": total_income,
            "total_expense": total_expense,
            "balance": balance
        }

    def to_list(self):
        return [entry.to_list() for entry in self.incomes + self.expenses]

class Ledger:
    def __init__(self, csv_file):
        self.records = {}
        self.csv_file = csv_file
        self.load_from_csv()
    
    def add_income(self, amount, description):
        date = datetime.now().strftime('%Y-%m-%d')
        if date not in self.records:
            self.records[date] = DailyRecord(date)
        self.records[date].add_income(amount, description)
        self.save_to_csv()
    
    def add_expense(self, amount, description):
        date = datetime.now().strftime('%Y-%m-%d')
        if date not in self.records:
            self.records[date] = DailyRecord(date)
        self.records[date].add_expense(amount, description)
        self.save_to_csv()
    
    def get_daily_summary(self, date):
        if date in self.records:
            return self.records[date].get_summary()
        else:
            return None

    def get_all_summaries(self):
        return [record.get_summary() for record in self.records.values()]

    def save_to_csv(self):
        with open(self.csv_file, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['date', 'amount', 'description', 'type', 'cumulative_income', 'cumulative_expense', 'running_balance'])
            
            for record in self.records.values():
                cumulative_income = 0
                cumulative_expense = 0
                for entry in record.incomes:
                    cumulative_income += entry.amount
                    running_balance = cumulative_income - cumulative_expense
                    writer.writerow([entry.date, entry.amount, entry.description, entry.entry_type, cumulative_income, cumulative_expense, running_balance])
                for entry in record.expenses:
                    cumulative_expense += entry.amount
                    running_balance = cumulative_income - cumulative_expense
                    writer.writerow([entry.date, entry.amount, entry.description, entry.entry_type, cumulative_income, cumulative_expense, running_balance])

    def load_from_csv(self):
        try:
            with open(self.csv_file, 'r') as file:
                reader = csv.reader(file)
                next(reader)  # Skip header
                for row in reader:
                    date, amount, description, entry_type, cumulative_income, cumulative_expense, running_balance = row
                    amount = float(amount)
                    cumulative_income = float(cumulative_income)
                    cumulative_expense = float(cumulative_expense)
                    running_balance = float(running_balance)
                    
                    if date not in self.records:
                        self.records[date] = DailyRecord(date)
                    
                    if entry_type == 'income':
                        self.records[date].add_income(amount, description)
                    elif entry_type == 'expense':
                        self.records[date].add_expense(amount, description)
        except FileNotFoundError:
            # No CSV file found, nothing to load
            pass

In [160]:
# Example usage
ledger = Ledger('financial_records.csv')
ledger.add_income(3438, 'Saveing')
ledger.add_expense(65, 'Lunch')
ledger.add_expense(65, 'Diner')

summary = ledger.get_daily_summary(datetime.now().strftime('%Y-%m-%d'))
print(summary)

all_summaries = ledger.get_all_summaries()
print(all_summaries)

{'date': '2024-07-19', 'total_income': 3438, 'total_expense': 130, 'balance': 3308}
[{'date': '2024-07-19', 'total_income': 3438, 'total_expense': 130, 'balance': 3308}]


In [161]:
import pandas as pd
df = pd.read_csv('financial_records.csv')
df

,date,amount,description,type,cumulative_income,cumulative_expense,running_balance
0,2024-07-19,3438,Saveing,income,3438,0,3438
1,2024-07-19,65,Lunch,expense,3438,65,3373
2,2024-07-19,65,Diner,expense,3438,130,3308
